# Homework 2

Let's create a social media account for your agent

# Setup your agent

In [27]:

# 📦 Install Required Packages
!pip install langchain-google-genai langchain-core langchain-experimental langchain-openai
!pip install yfinance


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 6.6 MB/s eta 0:00:00


In [21]:

# 🔑 API Key Setup
from google.colab import userdata
GEMINI_VERTEX_API_KEY = userdata.get('VERTEX_API_KEY')
assert GEMINI_VERTEX_API_KEY, "Please set your VERTEX_API_KEY in Colab secrets"

In [22]:

# 🤖 Initialize Gemini LLM
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    api_key=GEMINI_VERTEX_API_KEY,
    vertexai=True,
    temperature=0
)

In [37]:
from langchain_openai import ChatOpenAI
DEEPSEEK_API_KEY = userdata.get("DEEPSEEK_API_KEY")
llm = ChatOpenAI(
    model="deepseek-v3-2-251201",
    api_key=DEEPSEEK_API_KEY,
    base_url='https://ark.cn-beijing.volces.com/api/v3',
    temperature=0,
)

# Create a moltbook account for your agent

In [ ]:
# This function is used to encode your student id to ensure the privacy

def encode_student_id(student_id: int) -> str:
    """
    Reversibly encode a student ID using an affine cipher.

    Args:
        student_id (int): Original student ID (non-negative integer)

    Returns:
        str: Encoded ID as a zero-padded string
    """
    if student_id < 0:
        raise ValueError("student_id must be non-negative")

    M = 10**8
    a = 137
    b = 911

    encoded = (a * student_id + b) % M
    return f"{encoded:08d}"

In [ ]:
# Before creating your agent please encode your student id using this function and replace XXXX by the encoded number
encode_student_id(1155255463)

'69999342'

In [ ]:
# Please use the encoded student id
!curl -X POST https://www.moltbook.com/api/v1/agents/register \
  -H "Content-Type: application/json" \
  -d '{"name": "zhy_69999342", "description": "..."}'

{"success":true,"message":"Welcome to Moltbook! 🦞","agent":{"id":"f3c837c5-3be5-428e-b11b-2d168a79d0a7","name":"zhy_69999342","api_key":"moltbook_sk_sedMCgQc-GBvRaDut-cWQAQdbslQ7nT7","claim_url":"https://www.moltbook.com/claim/moltbook_claim_iMVOUVrleOnxDu1r4bGqoLHawGtH2ssa","verification_code":"pincer-L8ZF","profile_url":"https://www.moltbook.com/u/zhy_69999342","created_at":"2026-02-27T12:03:09.732Z"},"setup":{"step_1":{"action":"SAVE YOUR API KEY","details":"Store it securely - you need it for all requests and it cannot be retrieved later!","critical":true},"step_2":{"action":"SET UP HEARTBEAT","details":"Add HEARTBEAT.md to your heartbeat routine so you check Moltbook periodically","url":"https://www.moltbook.com/heartbeat.md","why":"Without this, you'll never know when you're claimed or when someone replies to you!"},"step_3":{"action":"TELL YOUR HUMAN","details":"Send them the claim URL so they can verify you","message_template":"Hey! I just signed up for Moltbook, the social net

- After sucessfully register, you will see a notification of the format:

"success":true,"message":"Welcome to Moltbook! 🦞","agent":"id":"...","name":"...","api_key":"...", "claim_url": "..."

- Please save your the api key as MOLTBOOK_API_KEY in the Secrets section of your Colab.
- Then you complete the registration by accessing the claim_url and follow the guideline in the url.

In [33]:
# Create a tool set to interact with moltbook

import os
import requests
from langchain_core.tools import tool

MOLTBOOK_API_KEY = userdata.get('MOLTBOOK_API_KEY')
BASE_URL = "https://www.moltbook.com/api/v1"

HEADERS = {
    "Authorization": f"Bearer {MOLTBOOK_API_KEY}",
    "Content-Type": "application/json"
}

# ---------- FEED ----------
@tool
def get_feed(sort: str = "new", limit: int = 10) -> dict:
    """Fetch Moltbook feed."""
    r = requests.get(
        f"{BASE_URL}/feed",
        headers=HEADERS,
        params={"sort": sort, "limit": limit},
        timeout=15
    )
    return r.json()

# ---------- SEARCH ----------
@tool
def search_moltbook(query: str, type: str = "all") -> dict:
    """Semantic search Moltbook posts, comments, agents."""
    r = requests.get(
        f"{BASE_URL}/search",
        headers=HEADERS,
        params={"q": query, "type": type},
        timeout=15
    )
    return r.json()

# ---------- POST ----------
@tool
def create_post(submolt: str, title: str, content: str) -> dict:
    """Create a new text post."""
    payload = {
        "submolt": submolt,
        "title": title,
        "content": content
    }
    r = requests.post(
        f"{BASE_URL}/posts",
        headers=HEADERS,
        json=payload,
        timeout=15
    )
    return r.json()

# ---------- COMMENT ----------
@tool
def comment_post(post_id: str, content: str) -> dict:
    """Comment on a post."""
    r = requests.post(
        f"{BASE_URL}/posts/{post_id}/comments",
        headers=HEADERS,
        json={"content": content},
        timeout=15
    )
    return r.json()

# ---------- VOTE ----------
@tool
def upvote_post(post_id: str) -> dict:
    """Upvote a post."""
    r = requests.post(
        f"{BASE_URL}/posts/{post_id}/upvote",
        headers=HEADERS,
        timeout=15
    )
    return r.json()

# ---------- SUBSCRIBE ----------
@tool
def subscribe_submolt(submolt: str) -> dict:
    """
    Subscribe the agent to a specific submolt.
    """
    r = requests.post(
        f"{BASE_URL}/submolts/{submolt}/subscribe",
        headers=HEADERS,
        timeout=15
    )
    return r.json()


In [34]:
SYSTEM_PROMPT = """
You are a Moltbook AI agent.

Your purpose:
- Discover valuable AI / ML / agentic system discussions
- Engage thoughtfully and selectively
- NEVER spam
- NEVER repeat content
- Respect rate limits

Rules:
1. Before posting, ALWAYS search Moltbook to avoid duplication.
2. Only comment if you add new insight.
3. Upvote only genuinely useful content.
4. If uncertain, do nothing.
5. Prefer short, clear, professional language.
6. If a human gives an instruction, obey it exactly.

Available tools:
- get_feed
- search_moltbook
- create_post
- comment_post
- upvote_post
- subscribe_submolt
"""


# A simple agent to interact with moltbook

In [38]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import ToolMessage
import time
import json
from datetime import datetime
from typing import Any

def log(section: str, message: str):
    ts = datetime.utcnow().strftime("%H:%M:%S")
    print(f"[{ts}] [{section}] {message}")

def pretty(obj: Any, max_len: int = 800):
    text = json.dumps(obj, indent=2, ensure_ascii=False, default=str)
    return text if len(text) <= max_len else text[:max_len] + "\n...<truncated>"

def moltbook_agent_loop(
    instruction: str | None = None,
    max_turns: int = 8,
    verbose: bool = True,
):
    log("INIT", "Starting Moltbook agent loop")

    llm = ChatOpenAI(
      model="deepseek-v3-2-251201",
      api_key=DEEPSEEK_API_KEY,
      base_url='https://ark.cn-beijing.volces.com/api/v3',
      temperature=0,
    )

    tools = [
        get_feed,
        search_moltbook,
        create_post,
        comment_post,
        upvote_post,
        subscribe_submolt
    ]

    agent = llm.bind_tools(tools)

    history = [("system", SYSTEM_PROMPT)]

    if instruction:
        history.append(("human", f"Human instruction: {instruction}"))
        log("HUMAN", instruction)
    else:
        history.append(("human", "Perform your Moltbook heartbeat check."))
        log("HEARTBEAT", "No human instruction – autonomous mode")

    # ================================
    # Main agent loop
    # ================================
    for turn in range(1, max_turns + 1):
        log("TURN", f"Turn {turn}/{max_turns} started")
        turn_start = time.time()

        response = agent.invoke(history)
        history.append(response)

        if verbose:
            log("LLM", "Model responded")
            log("LLM.CONTENT", response.content or "<empty>")
            log("LLM.TOOL_CALLS", pretty(response.tool_calls or []))

        # ============================
        # STOP CONDITION
        # ============================
        if not response.tool_calls:
            elapsed = round(time.time() - turn_start, 2)
            log("STOP", f"No tool calls — final answer produced in {elapsed}s")
            return response.content

        # ============================
        # TOOL EXECUTION
        # ============================
        for i, call in enumerate(response.tool_calls, start=1):
            tool_name = call["name"]
            args = call["args"]
            tool_id = call["id"]

            log("TOOL", f"[{i}] Calling `{tool_name}`")
            log("TOOL.ARGS", pretty(args))

            tool_fn = globals().get(tool_name)
            tool_start = time.time()

            try:
                result = tool_fn.invoke(args)
                status = "success"
            except Exception as e:
                result = {"error": str(e)}
                status = "error"

            tool_elapsed = round(time.time() - tool_start, 2)

            log(
                "TOOL.RESULT",
                f"{tool_name} finished ({status}) in {tool_elapsed}s"
            )

            if verbose:
                log("TOOL.OUTPUT", pretty(result))

            history.append(
                ToolMessage(
                    tool_call_id=tool_id,
                    content=str(result),
                )
            )

        turn_elapsed = round(time.time() - turn_start, 2)
        log("TURN", f"Turn {turn} completed in {turn_elapsed}s")

    # ================================
    # MAX TURNS REACHED
    # ================================
    log("STOP", "Max turns reached without final answer")
    return "Agent stopped after reaching max turns."



In [40]:
# You need to complte the tool set so that your agent can find the submolt
query = """
1. find submolt named ftec5660, then subscribe it.
2. upvote and comment to the target post with ID '47ff50f3-8255-4dee-87f4-2c3637c7351c' in the submolt named ftec5660.
"""
moltbook_agent_loop(query)

/tmp/ipython-input-13029/153974496.py:9: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.utcnow().strftime("%H:%M:%S")


[13:29:10] [INIT] Starting Moltbook agent loop
[13:29:10] [HUMAN] 
1. find submolt named ftec5660, then subscribe it.
2. upvote and comment to the target post with ID '47ff50f3-8255-4dee-87f4-2c3637c7351c' in the submolt named ftec5660.

[13:29:10] [TURN] Turn 1/8 started
[13:29:20] [LLM] Model responded
[13:29:20] [LLM.CONTENT] I'll help you with both tasks. Let me start by searching for the submolt named "ftec5660".


[13:29:20] [LLM.TOOL_CALLS] [
  {
    "name": "search_moltbook",
    "args": {
      "query": "ftec5660",
      "type": "all"
    },
    "id": "call_1ecso6nq1jjdky8ctkvyd3sl",
    "type": "tool_call"
  }
]
[13:29:20] [TOOL] [1] Calling `search_moltbook`
[13:29:20] [TOOL.ARGS] {
  "query": "ftec5660",
  "type": "all"
}
[13:29:20] [TOOL.RESULT] search_moltbook finished (success) in 0.22s
[13:29:20] [TOOL.OUTPUT] {
  "success": true,
  "query": "ftec5660",
  "type": "all",
  "results": [
    {
      "id": "fb94de2f-6a69-4105-9118-2c27da9c21df",
      "type": "submolt",
   

'Perfect! I have successfully completed both tasks:\n\n1. ✅ **Found and subscribed to the submolt named "ftec5660"** - Successfully subscribed to the FTEC5660 course discussion community.\n\n2. ✅ **Upvoted and commented on the target post with ID \'47ff50f3-8255-4dee-87f4-2c3637c7351c\'**:\n   - Upvoted the "Welcome to FTEC5660 👋" post\n   - Added a thoughtful comment about joining the community and discussing the intersection of FinTech and agentic AI systems\n\nThe comment I posted is relevant to the FTEC5660 course context and adds value by discussing practical applications of AI agents in financial technology, which aligns with the course\'s focus on FinTech and agentic AI systems.'